In [ ]:
import json
import pickle
import networkx as nx
import chexmix.graph as graph
from chexmix.utils import remove_symbols

In [ ]:
## KPEB data load
with open('../data/KPEB/KPEB_name_taxID.json', 'r') as f:
    kpeb_data = json.load(f)
kpeb_data = {int(k): v for k,v in kpeb_data.items()}

##viridiplantae load
with open('../data/viridiplanae.txt', 'rb') as f:
    viridiplantae = pickle.load(f)
path = '../data/network/'

In [ ]:
parent_node_type = graph.TaxParentType.Genus # Recommend Genus
keyword = 'amentoflavone'

In [ ]:
file_name = path + remove_symbols(keyword)
pubmed_graph = graph.PubMedGraph.from_keyword(keyword) ##
article_ids = pubmed_graph.get_article_ids()
pubtator_graph = graph.PubTatorGraph.from_article_ids(article_ids)

In [ ]:
tax_graph = graph.TaxonomyGraph.from_pubtator_bioentities(parent_node_type, pubtator_graph.get_bioentities('TAXO'),
                                                          viridiplantae, 'KPEB', kpeb_data)

In [ ]:
biograph_of_keyword = graph.BioGraph()
biograph_of_keyword.add_edges_from([(keyword, genus) for genus in tax_graph.find_roots()])
biograph_of_keyword = biograph_of_keyword.union(tax_graph)
biograph_of_keyword = biograph_of_keyword.inherit_attr_from(tax_graph)
biograph_of_keyword = biograph_of_keyword.inherit_attr_from(pubmed_graph)
nx.write_graphml(biograph_of_keyword, f'{file_name}.graphml')